In [1]:

from pyspark import SparkContext
sc = SparkContext(appName = "exercise test")


In [2]:


# 1) What are thelowestandhighesttemperaturesmeasuredeachyearfortheperiod1950-2014.
# Provide the lists sorted in the descending order with respect to the maximum temperature. In
# this exercise you will use the temperature-readings.csv file

def max_temperatures(a,b):
    if a>=b:
        return a 
    else:
        return b
    
temperature_file = sc.textFile("temperature-readings-small.csv")

lines = temperature_file.map(lambda line: line.split(";"))

year_temperature = lines.map(lambda x:(int(x[1][0:4]),float(x[3])))\
                        .filter(lambda x: int(x[0])>1950 or int(x[0]<2014))
                        
max_temps = year_temperature.reduceByKey(max)
min_temps = year_temperature.reduceByKey(min)

combine_rdd = max_temps.union(min_temps)\
                        .sortBy(lambda x:x[1],ascending=False)

combine_rdd.take(10)


[(2014, 29.1), (2013, 10.2), (2013, -13.3), (2014, -24.3)]

In [3]:
monthly_reading = lines.map(lambda x:(x[1][0:7],float(x[3])))\
                        .filter(lambda x: (x[1] >= 10))\
                        .map(lambda x: (x[0],1))\
                        .reduceByKey(lambda x,y: x+y)\
                        .map(lambda x: (int(x[0][0:4]),int(x[0][5:7]),x[1]))
                        
                        
monthly_reading.take(10)

[(2014, 5, 31),
 (2014, 6, 54),
 (2014, 8, 57),
 (2014, 9, 25),
 (2013, 11, 1),
 (2014, 4, 9),
 (2014, 7, 62),
 (2014, 10, 18),
 (2014, 11, 2)]

In [4]:
monthly_reading = lines.map(lambda x:(x[1][0:7],float(x[3])))\
                        .filter(lambda x: (x[1] >= 10))\
                        .map(lambda x: (x[0],1))\
                        .reduceByKey(lambda x,y: x+y)\
                        .map(lambda x: (int(x[0][0:4]),int(x[0][5:7]),1))


monthly_reading.take(10)

[(2014, 5, 1),
 (2014, 6, 1),
 (2014, 8, 1),
 (2014, 9, 1),
 (2013, 11, 1),
 (2014, 4, 1),
 (2014, 7, 1),
 (2014, 10, 1),
 (2014, 11, 1)]

In [5]:
#def 

year_temperature = lines.map(lambda x:(int(x[0]),int(x[1][0:4]),int(x[1][5:7]),float(x[3])))\
                        .filter(lambda x:(x[1]>1960 or x[2]<2014))\
                        .map(lambda x:((str(x[0]) +","+ str(x[1]) +","+ str(x[2])),float(x[3])))\
                        .reduceByKey(lambda x,y: (x + y)/2)\
                        .map(lambda x:(int(x[0][0:6]),int(x[0][7:11]),int(x[0][12:]),float(x[1])))

    
year_temperature.collect()

[(102170, 2013, 11, -2.0003417526335583),
 (102170, 2014, 2, 1.922969836488312),
 (102170, 2014, 3, 4.069735469970217),
 (102170, 2014, 5, 17.39750592222906),
 (102170, 2014, 6, 15.63130711000733),
 (102170, 2014, 9, 3.522406336468481),
 (102170, 2014, 10, -0.7295115266715948),
 (102170, 2014, 12, -8.169879193493859),
 (102170, 2013, 12, 2.6110939705074534),
 (102170, 2014, 1, -4.70177628986221),
 (102170, 2014, 4, 3.1893467007620595),
 (102170, 2014, 7, 17.137965973876884),
 (102170, 2014, 8, 11.971542739182839),
 (102170, 2014, 11, -1.3014219025835239)]

In [7]:
temperature_file = sc.textFile("temperature-readings-small.csv")
lines = temperature_file.map(lambda line: line.split(";"))
preci_file = sc.textFile("precipitation-readings.csv")

In [8]:
temp_2 = lines.map(lambda x: ((x[0] +","+x[1]),float(x[3])))\
                .filter(lambda x: (x[1] > 25 and x[1] < 30))\
                .reduceByKey(max)

temp_2.take(10)

[('102170,2014-07-10', 26.5),
 ('102170,2014-07-21', 27.6),
 ('102170,2014-07-23', 29.1),
 ('102170,2014-07-09', 28.2),
 ('102170,2014-07-22', 28.0),
 ('102170,2014-07-26', 28.7),
 ('102170,2014-07-29', 25.5)]

In [9]:
preci_1 = preci_file.map(lambda line: line.split(";"))\
                    .map(lambda x: ((x[0] + "," +  x[1]),float(x[3])))\
                    .filter(lambda x: (x[1] > 0.01 and x[1] < 0.2))\
                    .reduceByKey(max)

preci_1.take(10)

[('103100,1996-01-11', 0.1),
 ('103100,1996-02-10', 0.1),
 ('103100,1996-05-14', 0.1),
 ('103100,1996-09-13', 0.1),
 ('103100,1996-11-02', 0.1),
 ('103100,1996-12-03', 0.1),
 ('103100,1996-12-12', 0.1),
 ('103100,1996-12-30', 0.1),
 ('103100,1997-02-11', 0.1),
 ('103100,1997-02-20', 0.1)]

In [10]:
combine_rdd = temp_2.union(preci_1)\
                    .reduceByKey(lambda x,y: (x,y))
combine_rdd.take(10)

[('103100,1996-01-11', 0.1),
 ('103100,1999-01-01', 0.1),
 ('103100,2006-05-02', 0.1),
 ('103100,2006-06-22', 0.1),
 ('103100,2010-06-20', 0.1),
 ('103100,2011-10-28', 0.1),
 ('103100,2012-02-19', 0.1),
 ('103100,2013-10-10', 0.1),
 ('103100,2014-03-26', 0.1),
 ('103100,2014-04-05', 0.1)]

In [271]:
station_number = sc.textFile("stations-Ostergotland.csv")\
                     .map(lambda line: line.split(";"))\
                    .map(lambda x: x[0])
station_bc  = sc.broadcast(station_number.collect())

preci_2 = preci_file.map(lambda line: line.split(";"))\
                    .filter(lambda x:x[0] in station_bc.value)\
                    .map(lambda x: ((int(x[0]),x[1][0:7]),float(x[3])))\
                    .reduceByKey(lambda x,y:(x+y)/2)\
                    .map(lambda x:(x[0][0],x[0][1],x[1]))

In [272]:
preci_2.take(10)

[(75520, '1996-02', 3.814697265625022e-06),
 (75520, '1997-08', 9.5367431640625e-08),
 (75520, '1999-08', 0.003125593066215515),
 (75520, '2001-09', 3.4694469519536144e-19),
 (75520, '2003-07', 2.5861662813976734e-11),
 (75520, '2004-08', 1.587904989719391e-08),
 (75520, '2004-11', 3.706445415378645e-13),
 (75520, '2008-04', 0.1),
 (75520, '2008-08', 1.4901161193847657e-09),
 (75520, '2008-12', 1.5336912488972032e-52)]